In [1]:
import os
import pandas
#import itertools
#import nibabel as ni
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
#from statsmodels.sandbox.stats.multicomp import multipletests
#from nilearn import image, input_data

In [ ]:
# DOWNLOAD INSTRUCTIONS

In [ ]:
# Inititalize data

aba_dir = '/data1/users/jvogel/Allen_Human_Brain_Atlas/'
probes = pandas.read_csv(os.path.join(aba_dir,'normalized_microarray_donor9861/Probes.csv'))

In [ ]:
# GET EVERYTHING INTO A GIANT DATAFRAME AND SAVE DONOR INFORMATION
bigsheet = []
xpsheets = sorted(glob(os.path.join(aba_dir,
                                    'normalized_microarray_donor*/MicroarrayExpression.csv'
                                   )))
dids = [x.split('/')[-2].split('donor')[-1] for x in xpsheets]
for sheet in xpsheets:
    did = sheet.split('/')[-2].split('donor')[-1] 
    gxp = pandas.read_csv(sheet,header=None)
    gxp.drop(gxp.columns[0],axis=1,inplace=True)
    #gxp = gxp.loc[mprobes.index]
    
    # dummy variables for donor
    for tid in dids:
        if tid == did:
            gxp.loc['is_%s'%tid] = 1
        else:
            gxp.loc['is_%s'%tid] = 0
    bigsheet.append(gxp)
    print('finished',did)
    
bigdf = pandas.concat(bigsheet,axis=1).transpose()

In [ ]:
# PREP FOR STATISTICAL TESTING
bigdf.columns = ['col_%s'%x for x in bigdf.columns[:-6]] + bigdf.columns[-6:].tolist()
resid_df = pandas.DataFrame(np.empty_like(bigdf.values),
                            index = bigdf.index, columns=bigdf.columns)
resid_df.iloc[:,-6:] = bigdf.iloc[:,-6:].values

In [ ]:
# RUN STATS (took about a minute on the work computer)
for i,col in enumerate(bigdf.columns[:-6]):
    resid_df.loc[:,col] = smf.ols('%s ~ is_10021 + is_12876 + is_14380 + is_15496 + is_15697'%(col), 
                                  data=bigdf).fit().resid
    print(i)

In [ ]:
# SAVE BACK INTO SPREADSHEETS
dfz = {}
for col in resid_df.columns[-6:]:
    did = col.split('_')[-1]
    #ddir = '/data1/users/jvogel/Allen_Human_Brain_Atlas/normalized_microarray_donor%s'%did
    refsht = os.path.join(ddir,'MicroarrayExpression.csv')
    print('loading reference csv')
    gxp = pandas.read_csv(refsht,header=None)
    gxp.drop(gxp.columns[0],axis=1,inplace=True)
    gxp = gxp.loc[mprobes.index]
    ndf = resid_df[resid_df[col]==1][resid_df.columns[:-6]].transpose()                            
    ndf.index = gxp.index                        
    ndf.columns = gxp.columns  
    print('saving new csv')
    #ndf.to_csv(os.path.join(ddir,'MExp_17k_genes_ctr_for_donor'))
    dfz.update({did: ndf})
    print('finished',did)